In [ ]:
from dotenv import load_dotenv
#import getpass
import os
# Load environment variables from the .secrets file (or .env file)
load_dotenv(dotenv_path='../../.secrets')
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["TAVILY_API_KEY"] = getpass.getpass()
#print(os.environ["TAVILY_API_KEY"])
#print(os.environ["LANGCHAIN_API_KEY"])
#print(os.environ["LANGCHAIN_ENDPOINT"])
#print(os.environ["ANTHROPIC_API_KEY"])

In [ ]:
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

https://hub.docker.com/r/ollama/ollama



docker pull ollama/ollama

docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

docker run -d --network=host -v open-webui:/app/backend/data -e OLLAMA_BASE_URL=http://127.0.0.1:11434 --name open-webui --restart always ghcr.io/open-webui/open-webui:main

docker compose up -d

docker volume create ollama-data
docker volume create open-webui-data

ollama models load llama2



curl -X POST http://localhost:11434/api/generate \
-H "Content-Type: application/json" \
-d '{"model": "tinyllama", "prompt": "What are birds?"}'

In [12]:
from langchain.llms.base import LLM
import requests
import json
from pydantic import BaseModel
from typing import Optional

class OllamaLLM(LLM, BaseModel):
    model: str = "tinyllama"
    base_url: str = "http://localhost:11434"

    @property
    def _llm_type(self) -> str:
        return "ollama"

    def _call(self, prompt: str, stop: Optional[list] = None) -> str:
       payload = {"model": self.model, "prompt": prompt}
       try:
           response = requests.post(
               f"{self.base_url}/api/generate",
               json=payload,
               stream=True,  # Enable streaming
           )
           response.raise_for_status()

           # Collect streamed responses
           full_response = ""
           for line in response.iter_lines(decode_unicode=True):
               if line:  # Skip empty lines
                   try:
                       data = json.loads(line)
                       #print("Streaming JSON Object:", data)  # Debugging
                       full_response += data.get("response", "")
                       if data.get("done", False):
                           break
                   except json.JSONDecodeError as e:
                       print(f"Failed to decode line: {line}. Error: {e}")
                       continue

           return full_response
       except requests.RequestException as e:
           raise ValueError(f"Error communicating with Ollama API: {e}")
       except ValueError as e:
           raise ValueError(f"Error processing the response: {e}")



# Initialize the Ollama LLM
ollama_llm = OllamaLLM()

# Example usage
prompt = "What are birds?"
response = ollama_llm(prompt)
print("Response:", response)


Response: Birds are vertebrate animals that possess feathers and a fully developed beak, which allows them to feed on plant material through the process of nectar collection and seed dispersal. Birds come in all shapes and sizes, from small songbirds like robins and sparrows, to large birds such as eagles, owls, and vultures.

In addition to their wings and feet, birds have feathered plumes or a beak at the end of their bills that they use to catch prey in water, on land, or in trees. Some birds also have sharp talons that help them grasp their food or protect themselves from predators. Overall, birds are highly specialized animals that require a diverse range of adaptations for their diverse environments.


In [16]:
import json
import requests
from typing import Optional
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import LLMResult
from langchain.llms.base import LLM

class OllamaLLM(LLM):
    """
    A custom LLM integration for Ollama's API.
    """

    model: str = "tinyllama"  # Default model
    base_url: str = "http://localhost:11434"  # Default Ollama endpoint

    def _call(self, prompt: str, stop: Optional[list] = None) -> str:
        """
        Handles the interaction with the Ollama API.
        """
        payload = {"model": self.model, "prompt": prompt}
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json=payload,
                stream=True,  # Enable streaming
            )
            response.raise_for_status()

            # Process streamed responses
            full_response = ""
            for line in response.iter_lines(decode_unicode=True):
                if line:  # Skip empty lines
                    try:
                        data = json.loads(line)
                        #print("Streaming JSON Object:", data)  # Debugging
                        full_response += data.get("response", "")
                        if data.get("done", False):  # Stop when done
                            break
                    except json.JSONDecodeError as e:
                        print(f"Failed to decode line: {line}. Error: {e}")
                        continue

            return full_response
        except requests.RequestException as e:
            raise ValueError(f"Error communicating with Ollama API: {e}")
        except ValueError as e:
            raise ValueError(f"Error processing the response: {e}")

    @property
    def _identifying_params(self) -> dict:
        """Returns identifying parameters for serialization."""
        return {"model": self.model, "base_url": self.base_url}

    @property
    def _llm_type(self) -> str:
        """Type of the LLM."""
        return "ollama"

# Instantiate the Ollama LLM
ollama_llm = OllamaLLM(model="tinyllama", base_url="http://localhost:11434")

# Create a PromptTemplate
prompt = PromptTemplate(
    input_variables=["topic"],  # Variables to inject
    template="Explain {topic} in simple terms.",
)

# Create an LLMChain
#chain = LLMChain(llm=ollama_llm, prompt=prompt)
#result = chain.run(topic="maths")
chain = prompt | ollama_llm
# Run the chain with a specific input
result = chain.invoke({"topic": "math"})
print("LangChain Response:", result)


LangChain Response: Math is a subject that many people find difficult to understand, and for good reason. But don't worry - we're here to help! Here are some basic math terms and concepts that should make it easier to grasp the basics of mathematics:

1. Math symbols: There are a few commonly used symbols in mathematics, including the letter 'x', which represents an unknown variable or part of a problem (e.g., x is missing from the equation above). Other common symbols include '=', '+', '-', '*', '/', and '(', which represents multiplication and division, respectively.

2. Addition: When we add two numbers together, we're adding each number to the left of '+' (or to the right if it's negative). So for example, 4 + 8 is 12.

3. Subtraction: To subtract one number from another, we're taking away each number to the left of '-' (or to the right if it's positive). So for example, 9 - 2 is 7.

4. Multiplication: When we multiply two numbers together, we're multiplying each number to the righ